<a href="https://colab.research.google.com/github/dd5nt/Counters/blob/main/Main09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Загрузка библиотек
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Concatenate,Dense, Activation, Dropout, BatchNormalization, concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import Precision, Recall
from keras.preprocessing.text import Tokenizer, text_to_word_sequence,tokenizer_from_json

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import os

In [ ]:
# @title Загрузка датасета
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
!ls "gdrive/MyDrive/Project2/Data/"
!cp -r /content/gdrive/MyDrive/Project2/Data	.
#!cp -r /content/gdrive/MyDrive/Project2/Photo	Photo
#z= zipfile.ZipFile('/content/Data/Reestr.zip','r')
!unzip -q -o '/content/Data/Reestr.zip' -d '/content/Data/'
!ls "/content/Data/"
df = pd.read_excel(open('/content/Data/Reestr.xlsx','rb'))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Image  Reestr.zip
Image  Reestr.xlsx  Reestr.zip


In [ ]:
IMG_WIDTH = 768         # Ширина картинки
IMG_HEIGHT = 1024       # Высота картинки
TRAIN_DIR = 'train'     # Название папки с файлами обучающей выборки
VAL_DIR = 'val'         # Название папки с файлами проверочной выборки
TEST_DIR = 'test'       # Название папки с файлами тэстовой выборки
# Границы выборок
sep_train = int(0.8 * len(df))
sep_test = int(0.9 * len(df))
# Разбиение df на выборки
df_train = df.iloc[:sep_train]
df_val = df.iloc[sep_train+1:sep_test]
df_test = df.iloc[sep_test+1:]
print(df.shape)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(1929, 4)
(1543, 4)
(192, 4)
(192, 4)


In [ ]:
import os
import shutil
import pandas as pd

def load_imageset(df_load, folder_from, folder_to):
  dubl_file, error_file = 0, 0
  df_res = pd.DataFrame(columns=df_load.columns)
#  cur_time = time.time()
  for index, row in df_load.iterrows():
    file_name = Str(row['m_file'])+'.jpg'
    # Проверяем существование файла
    if os.path.isfile(folder_from + '/' + file_name):
      # Выполняем перенос файла
      shutil.move(folder_from + '/' + file_name, folder_to_to + '/' + file_name)
      df_res = df_res.append(row)
    else:
      if os.path.isfile(folder_to + '/' + file_name):
        print(f"Файл {file_name} уже перенесен в {folder_to}")
        df_res = df_res.append(row)
        dubl=+1
      else:
        print(f"Файл {file_name} не найден в {folder_from}")
        error_file=+1
  # Выводим статистику по переносу файлов
  total_files = len(df)
  success_files = len(found_df)
  failed_files = total_files - success_files
  print(f"Статистика:\n Всего {len(df_load)}\n Успешно перенесено: {len(df_res)}\n Не найдено файлов: {error_file}\n Дублирование файлов: {dubl_file}")
  results = df_res

def show_imageset(image_list, n):
    fig, axs = plt.subplots(1, n, figsize=(25, 10))
    for i in range(n):
        img = random.choice(image_list)
        axs[i].axis('off')
        axs[i].imshow(img)
    plt.show()

In [ ]:
import os
def create_folder(name):
  if os.path.exists(name):
    filelist = [f for f in os.listdir(name)]
    for f in filelist:
      os.remove(os.path.join(name, f))
    os.rmdir(name)
  os.mkdir(name)

In [ ]:
# Создаем структуру каталогов для обучения YOLO
path = os.getcwd()
root = path + '/meters/'
img_path = root + 'images/'
from_img = path + '/Data/Images/'
train_img = img_path + 'train/'
val_img = img_path + 'val/'
test_img = img_path + 'test/'
create_folder(train_img)
create_folder(val_img)
create_folder(test_img)

df_train = load_imageset(df_train, from_img, train_img )
#print(df_val.shape)
#print(df_test.shape)

NameError: ignored

In [ ]:
# Создаем структуру каталогов для обучения YOLO
path = os.getcwd()
root = path + '/meters/'
img_path = root + 'images/'
train_img = img_path + 'train/'
val_img = img_path + 'val/'
test_img = img_path + 'test/'
os.makedirs(train_img)
os.mkdir(val_img)
lbl_path = root + 'labels/'
train_lbl = lbl_path + 'train/'
val_lbl = lbl_path + 'val/'
test_lbl = lbl_path + 'test/'
os.makedirs(train_lbl)
os.mkdir(val_lbl)
# Обрабатываем полученные файлы
path = '/content/Image/'
# Папка с изображениями
images_path = path + 'images/'
# Читаем датасет
df = pd.read_csv(path + 'data.tsv', sep='\t')
# Получаем границу обучающей и проверочной выборки
sepVal = int(0.8 * len(df))
sepTest = int(0.1 * len(df))
df.head()

NameError: ignored

In [ ]:
def load_imageset(folder,subset,title):
    image_list = []
    cur_time = time.time()
    for filename in sorted(os.listdir(f'{folder}/{subset}')):
        image_list.append(image.load_img(os.path.join(f'{folder}/{subset}', filename),target_size=(IMG_WIDTH, IMG_HEIGHT)))
    print('{} выборка загружена. Время загрузки: {:.2f} с'.format(title, time.time() - cur_time))
    print('Количество изображений:', len(image_list))
    return image_list

def show_imageset(image_list, n):
    fig, axs = plt.subplots(1, n, figsize=(25, 10))
    for i in range(n):
        img = random.choice(image_list)
        axs[i].axis('off')
        axs[i].imshow(img)
    plt.show()

train_images = load_imageset(TRAIN_DIRECTORY, 'original', 'Обучающая')
val_images = load_imageset(VAL_DIRECTORY, 'original', 'Проверочная')

NameError: ignored